In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
!pip install numpy requests nlpaug
!pip install torch>=1.6.0 transformers>=4.0.0 sentencepiece
!pip install simpletransformers>=0.61.10

     |████████████████████████████████| 405 kB 5.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.20 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.4.1 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.28.0 which is incompatible.


In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
aug = naw.ContextualWordEmbsAug(
    model_path='distilbert-base-uncased', action="insert",aug_max=10,device='cuda')

In [3]:
data=pd.read_csv('/content/drive/MyDrive/project1/new_train.csv')

In [4]:
data

,Unnamed: 0,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe,Labels,encoded_cat,clean_comment_text
0,0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,Normal,0,explanation edits made username hardcore metal...
1,1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,Normal,0,aww match background colour seemingly stuck th...
2,2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,Normal,0,hey man really trying edit war guy constantly ...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,Normal,0,make real suggestion improvement wondered sect...
4,4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,Normal,0,sir hero chance remember page
...,...,...,...,...,...,...,...,...,...,...,...
159566,159566,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,Normal,0,second time asking view completely contradicts...
159567,159567,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,Normal,0,ashamed horrible thing put talk page
159568,159568,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,Normal,0,spitzer umm there actual article prostitution ...
159569,159569,And it looks like it was actually you who put ...,0,0,0,0,0,0,Normal,0,look like actually put speedy first version de...


In [5]:
rude_data=data[data['rude']==1]
rude_data

,Unnamed: 0,comment_text,malignant,highly_malignant,rude,threat,abuse,loathe,Labels,encoded_cat,clean_comment_text
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,"malignant,highly_malignant,rude,abuse",12,cocksucker piss around work
42,42,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1,"malignant,rude,abuse,loathe",24,gay antisemmitian archangel white tiger meow g...
43,43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0,"malignant,rude,abuse",23,fuck filthy mother as dry
51,51,GET FUCKED UP. GET FUCKEEED UP. GOT A DRINK T...,1,0,1,0,0,0,"malignant,rude",22,get fucked get fuckeeed got drink cant put get...
55,55,Stupid peace of shit stop deleting my stuff as...,1,1,1,0,1,0,"malignant,highly_malignant,rude,abuse",12,stupid peace shit stop deleting stuff asshole ...
...,...,...,...,...,...,...,...,...,...,...,...
159411,159411,Fat piece of shit \n\nyou obese piece of shit....,1,0,1,0,1,0,"malignant,rude,abuse",23,fat piece shit obese piece shit thought naked ...
159493,159493,FUCKING FAGGOT \n\nLOLWAT.,1,0,1,0,1,0,"malignant,rude,abuse",23,fucking faggot lolwat
159494,159494,"""\n\n our previous conversation \n\nyou fuckin...",1,0,1,0,1,1,"malignant,rude,abuse,loathe",24,previous conversation fucking shit eating libe...
159541,159541,Your absurd edits \n\nYour absurd edits on gre...,1,0,1,0,1,0,"malignant,rude,abuse",23,absurd edits absurd edits great white shark to...


In [ ]:
def threat(df):
    new_text=[]
    
    ##selecting the minority class samples
    df_n=df[df.threat==1].reset_index(drop=True)

    ## data augmentation loop
    for i in tqdm_notebook(range(0,len(df_n))):
        
            text = df_n.iloc[i]['clean_comment_text']
            augmented_text = aug.augment(text,n=31)
            for i in augmented_text:
              new_text.append(i)
    
    
    ## dataframe
    new=pd.DataFrame({'clean_comment_text':new_text,'threat':1})
    frames=[df,new]
    df=pd.concat(frames)
    df=df.fillna(0)
    return df

In [6]:
! pip install sentence-splitter

     |████████████████████████████████| 44 kB 1.5 MB/s 


In [7]:
! pip install transformers

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 3.3 MB 35.9 MB/s 
     |████████████████████████████████| 596 kB 36.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
! pip install SentencePiece

     |████████████████████████████████| 1.2 MB 6.2 MB/s 


In [9]:

import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)


Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [12]:
def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
df=threat_data['clean_comment_text'][0:20]

In [ ]:
for i in df:
  print(get_response(i,5))

In [13]:
def rude1(df):
    new_text=[]
    
    ##selecting the minority class samples
    df_n=df[df.rude==1].reset_index(drop=True)

    ## data augmentation loop
    for i in tqdm_notebook(range(0,len(df_n))):
        
            text = df_n.iloc[i]['clean_comment_text']
            augmented_text = get_response(text,1)
            for i in augmented_text:
              new_text.append(i)
    
    
    ## dataframe
    new=pd.DataFrame({'clean_comment_text':new_text,'rude':1})

    return new

In [ ]:
rude=rude1(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/8449 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3402: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


In [ ]:
rude

In [ ]:
rude.to_csv("/content/drive/MyDrive/project1/rude.csv")